# Overview

As I try to learn as much as I can about data science by working with publicly available data sets and the examples of others, the question of which set of data to analyze, given their relative ubiquity on the web nowadays, presented itself rather obstinantly to me.

So I thought of the current situation I find myself in, living abroad in the eternal city, Rome. If someone asks me what I like most of Rome, I must give pause and think of its infinite beautiful facades. But if someone asks me what I like least, I can answer quite easily: traffic. Rather than complaining, as one is wont to do in (southern(ish)) Italy, I decided that perhaps I could use my nascent learned data science skills to see if I can't learn something useful about Rome's traffic catastrophe and find patterns that only statistical methods on large data sets can find.

Sure enough, even Rome has certain public data free for the public to play with. So I obtained data on traffic accidents and traffic incidences involving pedestrians from Rome's public data repo at

http://dati.comune.roma.it/download

(all data is in Italian, of course) and did the most basic of analyses.

First we'll have a look at the data involving pedestrians ('pedoni') as the data set is smaller. Next we will have a look at the general traffic incidences, of which there are many more case with a plethora of descriptive data attached to each incidence.

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import json
import re
#from unidecode import unidecode
from sklearn import cross_validation
import datetime
import geoplotlib

/usr/lib/python3.5/site-packages/sklearn/utils/fixes.py:64: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  if 'order' in inspect.getargspec(np.copy)[0]:


SyntaxError: invalid syntax (__init__.py, line 295)

# Pedoni data

Here I took a much smaller data set that only involved pedestrians (and some sort of vehicle). This set was not as extensive as the general traffic incidence set below, but allowed me to warm up on reading in the JSON data Rome provided and look at the distribution of incidences as a function of age and gender, and to see what percentage of (reported) incidences result in a pedestrian death.



First we simply write a function to read in the data and clean it up a little bit so that Pandas can successfully parse the data.

In [2]:
def load_pedoni_data(json_file):
    with open(json_file, 'r') as filename:
        data = filename.readlines()

    # Overwrite first line with just a [    
    data[0] = '[\n'
    data[-1] = ']\n'

    with open('edited_json_pedoni.json', 'w') as filename:
        filename.writelines(data)

    return(pd.read_json("edited_json_pedoni.json", orient='records'))

pedoni = load_pedoni_data('json_pedoni_01.7.2014_31.12.2014.json')
pedoni.columns.values

array(['AANascita', 'Deceduto', 'DecedutoDopo', 'IDProtocollo', 'Sesso',
       'TipoLesione', 'TipoPersona'], dtype=object)

I want to look at the incidences as a function of age, since they provide the birth year in the data. I get rid of birth years of '0' (nonsensical data) then create a new column that gives the age of the victim.

In [ ]:
pedoni = pedoni.query('AANascita != 0')
pedoni['age'] = pedoni.apply(lambda row:2015-row['AANascita'], axis=1)

Now I'll plot the distribution of ages and gender in the incidences, comparing each distribution for incidences where the pedestrian survived versus where a fatality resulted (the 'Deceduto' feature indicates the victim died immediately as a result of the incident). We can also plot the number of incidences without fatalities versus those with.

In [ ]:
f, a = plt.subplots(1,3)
f.tight_layout()
f.set_size_inches(10,5)

width=0.35
index=np.arange(2)

a[0].hist(pedoni[pedoni.Deceduto == 'Falso'].age, color='b', alpha=0.45) # no fatality
# There is a problem with making a histogram of these data- perhaps not enough points
#a[0].hist(pedoni[pedoni.Deceduto == 'Vero'].age, color='r', alpha=0.65) # fatality
a[0].set_xlabel('Age')
a[0].set_ylabel('No. incidences')

plt.subplot2grid((1,3),(0,1))
pedoni[pedoni.Deceduto == 'Falso'].Sesso.value_counts().plot(kind='bar', color='b', alpha = 0.45) # no fatality
pedoni[pedoni.Deceduto == 'Vero'].Sesso.value_counts().plot(kind='bar', color='r', alpha = 0.65) # death resulted
plt.xlabel('Gender in incidence')
plt.ylabel('No. incidences')

plt.subplot2grid((1,3),(0,2))
pedoni.Deceduto.value_counts().plot(kind='bar', alpha = 0.45)
plt.xlabel('Death?')
plt.ylabel('No. incidences')

print('Total number of cases with fatalities: {0}'.format(len(pedoni[pedoni.Deceduto == 'Vero'])))

You can see in the distribution of the ages that the number of incidences with no fatalies far outweighs those with fatalities. So it isn't too useful to look at. However, in the traffic accidents below we can look at a more useful quantity- whether or not death *or* injury occurs, as this (macabre, yes) feature gives better statistics (and is still useful to look at).

# Incidenti data

We can proceed to analyze the traffic incident ('incidenti' in Italian) data, though we have to clean up the data a bit more thoroughly for processing, which needs to be done carefully, as the JSON files are much larger for these incidences (approximately 15K records for every 6 months of data). We examine one 6 month period in particular, July-December 2014 (the same period was used for the pedoni data above).

There are a few things we need to do first - in Python, so all processes are reproducible. The JSON file provided by the Roma public data utility has a few things we need to clear up- I remove the initial line with an unnecessary label of the file name and make sure brackets and commas are properly arranged so as to be read by pd.read_json. I also have to decode the text from 'latin' and then use a handy utility called 'unidecode' to reincode each record in the JSON file such that the accents (ubiquitous in Italian) are replaced with their nearest unaccented equivalent. 

I also remove one particular field, 'DaSpecificare' because it is a qualitative descriptive field (a short phrase describing the incident) that I don't expect to be of use in any analysis (it can't possibly be turned into a categorical variable) and because certain descriptions involved quotes in their descriptions (much like my current university isn't called La Sapienza, but rather Università di Roma, 'La Sapienza') that muck with the expected JSON format. So I just remove that field completely and write to a new file.

I also do a few other things that I have now lumped into this function "clean_incidenti_data", including replacing commas with decimals to match with the locale on my machine for converting to floats

In [ ]:
# for now pd.read_json seems to get angry if I load to many lines in, so only load 9000 for now 
# (just to get an idea of what to do- reuse same code with data set)
def load_incidenti_data(json_file):
    with open(json_file, 'r') as filename:
        # for now pd.read_json seems to get angry if I load to many lines in, so only load 9000 for now 
        # (just to get an idea of what to do- reuse same code with data set)
        #data = [next(filename) for x in range(5000)]
        data = filename.readlines()

    for num, line in enumerate(data,0):
        if num == 0:
            data[num] = '[\n'
        # must remove comma at end of last line
        elif num == len(data)-2 and len(data) > 2:
            # need to include a (mac?) symbol \r that comes with each new line
            tmp = re.sub("\"DaSpecificare.*?.\", ","",re.sub(",\r$","",line))
            # In the future I should look for a BOM to detect encoding automatically
            data[num] = unidecode(tmp.decode('latin'))
        elif num == len(data)-1:
            data[num] = ']\n'
        else:
            tmp = re.sub("\"DaSpecificare.*?.\", ","",line)
            data[num] = unidecode(tmp.decode('latin'))

    with open('edited_json_incidenti.json', 'w') as filename:
        filename.writelines(data)
    
    return(pd.read_json("edited_json_incidenti.json", orient='records'))

# We are overwriting the dataframe in each of the steps below, so we turn this warning off
pd.options.mode.chained_assignment = None  # default='warn'
def clean_incidenti_data(dataframe):
    dataframe['harmed'] = dataframe.apply(lambda row: row['NUM_MORTI']+row['NUM_FERITI'], axis=1)
    dataframe.drop(['NUM_MORTI','NUM_FERITI'], axis=1, inplace=True)

    # Drop terms that don't match the time format I want
    dataframe['time_match'] = dataframe.apply(lambda row: re.search(' \d{1,2}\.\d{2}',row['DataOraIncidente']) is not None, axis=1)
    dataframe = dataframe.query('time_match')

    # Gives the time (of day) as a time_struct 
    dataframe['time'] = dataframe.apply(lambda row: datetime.datetime.strptime(re.search(' \d{1,2}\.\d{2}\.\d{2}',row['DataOraIncidente']).group().lstrip(), '%H.%M.%S'), axis=1)
    dataframe.drop('time_match', inplace=True, axis=1)

    # Finally, convert European style decimals to US-style, since my locale is not European
    # A bit of a workaround, but it functions
    dataframe['Latitudine'] = dataframe.apply(lambda row: row['Latitudine'].replace(',','.'), axis=1)
    dataframe['Longitudine'] = dataframe.apply(lambda row: row['Longitudine'].replace(',','.'), axis=1)
    return(dataframe)

In [ ]:
with open('json_incidenti_01.7.2014_31.12.2014.json', 'r') as filename:
        data = filename.read().decode('latin')

Now before doing anything else, we set aside 30% of the data, randomly chosen, as test data. We take what is leftover as training data and clean it (which amounts to renaming some rows and converting European style decimals to American style for greater ease of the use of float).

In [ ]:
incidenti = load_incidenti_data('json_incidenti_01.7.2014_31.12.2014.json')

# test = incidenti.sample(frac=0.3, replace=False)
# train = incidenti.loc[~incidenti.index.isin(test.index)]
# train = clean_incidenti_data(train)

# train.columns.values

Let's look at distributions of events with people injured versus those without injury (or death). I've taken only the features that seem to have a reasonable number of categorical choices that might be used for fitting later.

In [ ]:
features=['CondizioneAtmosferica','FondoStradale','Illuminazione','Localizzazione1','Pavimentazione','ParticolaritaStrade','Segnaletica','TipoStrada','Traffico','Visibilita']
fig = plt.figure(figsize=(18,18), dpi=1600)
f, a = plt.subplots(len(features)/3+1,3,sharey=True,figsize=(17,15))
#f.tight_layout(pad=3)

for i,feature in enumerate(features,0):
    #print i, feature, i/2, i%3
    plt.subplot2grid((len(features)/3+1,3),(i%3,i/3))
    train[train.harmed == 0][feature].value_counts().plot(kind='bar', color='b', alpha=0.45)
    train[train.harmed > 0][feature].value_counts().plot(kind='bar', color='r', alpha=0.45)
    plt.xlabel(feature)

There don't seem to be any obvious deviations of incidences with harm done to those involved versus those without. For now we move on since there is no smoking gun discriminant between the two data sets in these categorical data.

We will look at latitude, longitude, and time features separately (they more warrant scatters and a histogram, respecively). We actually cut off the plot here because most of it is empty space- there are some outlier accidents under jurisdiction of Rome that should be removed from the data from reasonable analysis of the city proper.



In the two left plots we zoom in towards the city center. Later we will put these on a map.

In [ ]:
fig = plt.figure(figsize=(18,18), dpi=1600)
f, a = plt.subplots(2,3,sharey=True,figsize=(20,15))

# Effectively, this gives us a map of Rome with density of accidents
a[0,0].scatter(np.asarray(map(float,train.Latitudine[train.harmed == 0])),np.asarray(map(float,train.Longitudine[train.harmed == 0])), color='b', alpha = 0.45)
a[0,0].scatter(np.asarray(map(float,train.Latitudine[train.harmed > 0])),np.asarray(map(float,train.Longitudine[train.harmed > 0])), color='r', alpha = 0.45)
a[0,0].set_xlabel('Latitude')
a[0,0].set_ylabel('Longitude')

a[0,1].scatter(np.asarray(map(float,train.Latitudine[train.harmed == 0])),np.asarray(map(float,train.Longitudine[train.harmed == 0])), color='b', alpha = 0.45)
a[0,1].scatter(np.asarray(map(float,train.Latitudine[train.harmed > 0])),np.asarray(map(float,train.Longitudine[train.harmed > 0])), color='r', alpha = 0.45)
a[0,1].set_xlabel('Latitude')
a[0,1].set_xlim(41.4,42.4)
a[0,1].set_ylabel('Longitude')
a[0,1].set_ylim(12.2,12.8)

a[1,0].scatter(np.asarray(map(float,train.Latitudine[train.harmed == 0])),np.asarray(map(float,train.Longitudine[train.harmed == 0])), color='b', alpha = 0.45)
a[1,0].set_xlabel('Latitude')
a[1,0].set_xlim(41.8,41.95)
a[1,0].set_xticks(np.arange(41.8,41.95,.03))
a[1,0].set_ylabel('Longitude')
a[1,0].set_ylim(12.45,12.55)

a[1,1].scatter(np.asarray(map(float,train.Latitudine[train.harmed > 0])),np.asarray(map(float,train.Longitudine[train.harmed > 0])), color='r', alpha = 0.45)
a[1,1].set_xlabel('Latitude')
a[1,1].set_xlim(41.8,41.95)
a[1,1].set_xticks(np.arange(41.8,41.95,.03))
a[1,1].set_ylabel('Longitude')
a[1,1].set_ylim(12.45,12.55)

a[1,2].scatter(np.asarray(map(float,train.Latitudine[train.harmed == 0])),np.asarray(map(float,train.Longitudine[train.harmed == 0])), color='b', alpha = 0.45)
a[1,2].scatter(np.asarray(map(float,train.Latitudine[train.harmed > 0])),np.asarray(map(float,train.Longitudine[train.harmed > 0])), color='r', alpha = 0.45)
a[1,2].set_xlabel('Latitude')
a[1,2].set_xlim(41.8,41.95)
a[1,2].set_xticks(np.arange(41.8,41.95,.03))
a[1,2].set_ylabel('Longitude')
a[1,2].set_ylim(12.45,12.55)


We can also plot the incidences as a function of the time of day. Below we construct histograms for the two categories (no harm done in incident versus incidences with injury and/or death). 



[not yet]The second plot is the same as the first, but with the much cooler kernel density continuum extrapolation of the binned data.

In [ ]:
f, a = plt.subplots(2,1)
f.set_size_inches(10,7)
unharmed_bins = np.asarray(train[train.harmed == 0].time.groupby(train.time.dt.hour).count())
harmed_bins = np.asarray(train[train.harmed > 0].time.groupby(train.time.dt.hour).count())
ind = np.arange(24)
width = 1

a[0].bar(ind, unharmed_bins, width, color='b', alpha=0.45)
a[0].bar(ind, harmed_bins, width, color='r', alpha=0.45)
a[0].set_xticks(ind)
a[0].set_xlim(0,24)
a[0].set_xlabel('Hour of day')
a[0].set_ylabel('No. of incidences')
a[0].legend(loc='best')

# plt.subplot2grid((1,3),(1,0))
# np.asarray(train[train.harmed == 0].time.groupby(train.time.dt.hour).count()).plot(kind='kde')
# np.asarray(train[train.harmed == 0].time.groupby(train.time.dt.hour).count()).plot(kind='kde')
# come back to this

So one 'cool' thing we can see here ('cool' in the sense of patterns that we are familiar with being manifest, not in the sense of injury or death ocurring) is the bimodal peak in incidences at rush hour- around 9-10 AM and 4-6 PM (it's Rome, with Italian work hours, OK?). Unfortunately for discriminating against the two data sets it seems incidences involving death or injury do not follow any particularly distinct pattern from those that do not.

However, it is interesting to note that the only places where the number of incidences that result in injuries outnumber those that do not, at least in this data set, are late at night, between either 10-11 PM or 1-2 AM.

Now at this point I was thinking that this data looked a bit like the [Titanic Kaggle competition](http://nbviewer.ipython.org/github/agconti/kaggle-titanic/blob/master/Titanic.ipynb) data I had seen a little while ago- in the sense that there were many categorical features predicting, rather morbidly yet again, the likelihood of death in that particular row of data (in that case representing a passenger on the Titanic). 



So I will try something naive here that emulates that strategy. This means a logistic regression and then a fit based upon the more complicated (but still ridiculously easy to call here) support vector machine (SVM) supervised learning.

So maybe we can predict the likelihood of an accident occurring where I live when I typically walk home from work. I'll use some nearby coordinates and a time of about 19.30 of when I arrive home. We can try and compare this with...